[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/joshmaglione/CS102-Jupyter/main?labpath=.%2F11_Unsupervised.ipynb) 

<a href="https://colab.research.google.com/github/joshmaglione/CS102-Jupyter/blob/main/11_Unsupervised.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

[View on GitHub](https://github.com/joshmaglione/CS102-Jupyter/blob/main/11_Unsupervised.ipynb)

# 11 : Unsupervised learning

## Learning outcomes
By the end of this notebook, you should be able to:
- describe what unsupervised learning is and when it is useful
- apply dimensionality reduction (e.g., PCA) to simplify or visualize high-dimensional data
- interpret unsupervised results (components/clusters) and discuss limitations

![](https://miro.medium.com/v2/resize:fit:587/1*J82yf-YU7Ryhh5BdhyOdcw.png)

**What is the difference between supervised and unsupervised learning?**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

Unsupervised learning can highlight intrinsic information about the data.

One example is **dimension reduction**:
- take high-dimensional data and return lower-dimensional data that still retains most of the information (usually info = total variance)
- used for:
  - visualization (from 10 dimensions to 2)
  - speed up model training (from 2000 dimensions to 400)

## Principal component analysis

Principal component analysis (PCA) is perhaps one of the most broadly used unsupervised algorithms. 

(My personal favorite)

PCA is fundamentally a **dimensionality reduction** algorithm, but it can be used for (among other things)
- visualization
- noise filtering
- feature extraction

The mathematics around PCA is super fun. 

In [Geometric Foundations for Data Analysis](https://joshmaglione.com/2023CS4102.html) we go through PCA in detail. 

### Toy example

We compute the two principal components, which are just vectors.

We'll plot both principal components on the plot of the data.

In [ ]:
# Load the data
df = pd.read_csv("data/pcadata.csv")

# Helper function
def draw_vector(v0, v1, color='red'):
    ax = plt.gca()
    arrowprops = dict(arrowstyle='->', linewidth=2, shrinkA=0, shrinkB=0, color=color)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# Perform PCA
model = PCA(n_components=2)
model.fit(df)

# Plot the principal components
plt.grid()
plt.axis('equal')
plt.scatter(df.x, df.y, c='blue', alpha=0.5, zorder=2)
mu = model.mean_
PC1, PC2 = model.components_
var1, var2 = model.explained_variance_
draw_vector(mu, mu + 3*PC1, color='red')
draw_vector(mu, mu + 3*PC2, color='orange')
plt.text(mu[0] + 3*PC1[0] - 1, mu[1] + 3*PC1[1] + 0.5, 'PC1', color='red', fontsize=12, ha='right')
plt.text(mu[0] + 3*PC2[0] + 0.5, mu[1] + 3*PC2[1] + 0.75, 'PC2', color='orange', fontsize=12, ha='right')
plt.show()

In [ ]:
model = PCA(n_components=2)						# Bringing it here to see
model.fit(df)
print(f"First component: PC1 = {model.components_[0]}")
print(f"Second component: PC2 = {model.components_[1]}")

To each principal component is a value -- eigenvalue eigenvector pairs -- this measures variance.

The first principal component is where the largest variance occurs. 

It is decreasing (in order) for the rest of the principal components.

In [ ]:
# Project the data onto the first 2 principal components
projected_data = model.transform(df)

# Create a scatter plot of the projected data
plt.scatter(projected_data[:, 0], projected_data[:, 1], c='blue', alpha=0.5, zorder=2)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Data on First 2 Principal Components')
plt.grid()
plt.axis('equal')
plt.show()

Notice that PC1 is not just one variable $x$ or $y$. It is a *linear combination* of the two.

This can make it challenging to interpret the components, but for many scenarios this is OK.

We can get the vector of the explained variance. 

In [ ]:
model.explained_variance_ratio_

This tells us :
- $97.6\%$ of the variance is seen in PC1
- $2.4\%$ of the variance (the rest) is seen in PC2.

So really, we could just keep one dimension.

In [ ]:
pca = PCA(n_components=1)	# Only use 1 component
X = df.to_numpy()
pca.fit(X)
X_pca = pca.transform(X)
X_new = pca.inverse_transform(X_pca)
plt.grid()
plt.axis('equal')
plt.scatter(X[:, 0], X[:, 1], alpha=0.25)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.75)
plt.title('PCA with 1 Component')
plt.show()

The light blue points are the original data, while the orange points are the projected version. 

The key to PCA:
- information along the least important principal axes are removed!
- Th reduced dataset is, in some sense, good enough to encode the most important relationships between the points.

This is a toy example, so it's not easy to see the benefit, but it is easy to see the impact.

## Back to Iris

Let's load up the Iris data set again and plot the basic data.

In [ ]:
from sklearn.datasets import load_iris
import seaborn as sns

iris = load_iris()
ser = pd.Series(iris.target_names[iris.target], name='species')
df_labeled = pd.DataFrame(
	iris.data, 
	columns=iris.feature_names, 
)
df_labeled = pd.concat([ser, df_labeled], axis=1)

_ = sns.pairplot(
	df_labeled, 
	hue='species'
)	

(This is just an excuse to enjoy a nice image 🫠)

Let's also recall that we can get quick information with `describe`.

In [ ]:
df_labeled.describe()

In [ ]:
df_labeled.head()

Notice that the scale of petal width is much smaller than the other features. 

This is a problem for PCA, *which is sensitive to the scale of the features*. 

We can fix this by standardizing the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_labeled.drop('species', axis=1))
df_scaled = pd.DataFrame(df_scaled, columns=df_labeled.columns[1:])
df_scaled = pd.concat([df_labeled['species'], df_scaled], axis=1)
df_scaled.head()

Umm what?!

![](https://media.giphy.com/media/dAVLtOPb0JeIE/giphy.gif?cid=ecf05e476jqe1aysalqrm12p61ybtfb2bo4aeg27gpsrxng4&ep=v1_gifs_search&rid=giphy.gif&ct=g)

It's alright Troy, let's plot.

In [ ]:
_ = sns.pairplot(
	df_scaled, 
	hue='species'
)	

In [ ]:
df_scaled.describe()

The transformation is not a pure translation, but it does keep most of the "geometry" in tact. 

OK, let's actually do the PCA now.

In [ ]:
pca = PCA()
X = df_scaled.drop('species', axis=1)
X_new = pca.fit_transform(X)
X_new=pd.DataFrame(X_new, columns = [f"Feature {i}" for i in range(1,5)])
X_new.head()

In [ ]:
print("The explained variance of all 4 components:")
for i, r in enumerate(pca.explained_variance_ratio_):
	print(f"PC{i+1}: {r*100:.2f}%")

We would get over $95\%$ of the total variance in the first two components.

In [ ]:
sns.heatmap(X.corr(), annot = True, square = True)

In [ ]:
sns.heatmap(X_new.corr(), annot = True, square = True)

This is *the key feature* of PCA:
- the principal components are uncorrelated (linearly independent!).

So what are the principal components in this example?

In [ ]:
print("Principal components:")
for i, row in enumerate(pca.components_):
	print(f"PC{i+1}: {row}")

We might interpret "Feature 1" as approximately:
$$
	\frac{1}{2}x_{\text{s. len.}} - \frac{1}{4} x_{\text{s. wid.}} + \frac{1}{2} x_{\text{p. len.}} + \frac{1}{2} x_{\text{p. wid.}}
$$

Since the first two features have nearly all of the variances, let's project onto these two dimensions.

We'll plot the data.

In [ ]:
_ = sns.scatterplot(
	x=X_new['Feature 1'], 
	y=X_new['Feature 2'], 
	hue=df_scaled['species']
)

We could just include all 4 dimensions. Four is tiny, but two is nice to visualize.

Notice that PCA has not used the labels *and* you can see clusters of species.

## Clustering algorithms

Clustering algorithms group subsets of data based on features.

The notion of a "cluster" is tricky to define canonically. 

In one context clusters might be determined by
- distance from a centroid
- distance from nearest neighbor

There are many kinds of clustering algorithms. 

We will look at $k$-means clustering.

The idea behind $k$-means clustering:
- Specify $k$ the number of desired clusters
- It will find $k$ subsets of the data based on minimal distance to centroids.

If our data were in $\mathbb{R}^2$ and we wanted $k=9$ clusters, data gets partitioned based on the following *tesselation*.

![](imgs/Voronoi_growth_euclidean.gif)

Any data point in the blue region, for example, would be in the blue cluster. 

### Digits

We'll look at the `digits` data set in scikit learn. 

In [ ]:
from sklearn.datasets import load_digits

# Load the data
data, labels = load_digits(return_X_y=True)

# Printing aspects of the data
(n_samples, n_features) = data.shape
n_digits = np.unique(labels).size

print(f"Number of digits  : {n_digits}")
print(f"Number of samples : {n_samples}")
print(f"Number of features: {n_features}")

We have all ten digits represented in our data set. 

The number of data points (i.e. image data) is 1,797. 

Each data point lives in $\R^{64} \cong \R^{8\times 8}$.

In [ ]:
print(data[0].reshape(8, 8))

Below is some code to sample 240 digits and convert them into an $8\times 8$ image.

In [ ]:
digits = data.reshape(-1, 8, 8)
indices = np.random.choice(len(digits), size=240, replace=False)
sample_digits = digits[indices]

fig, axes = plt.subplots(12, 20, figsize=(10, 6))
for ax in axes.flatten():
    ax.tick_params(
        axis='both',
        which='both',
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False
    )
for i, ax in enumerate(axes.flatten()):
    ax.imshow(sample_digits[i], cmap='gray_r')

plt.show()

Think we need all 64 dimensions? Let's build code for PCA, but not use it.

In [ ]:
from sklearn.cluster import KMeans

pca = PCA(n_components=64)
reduced_data = pca.fit_transform(data)
kmeans = KMeans(n_clusters=n_digits)
kmeans.fit(reduced_data)
for i, lab in enumerate(np.unique(kmeans.labels_)):
	print(f"Cluster {i}: {np.sum(kmeans.labels_ == lab)} samples")

There are more parameters that we can tune for $k$-means, but we're ignoring that.

Also note that "Cluster $k$" is not necessarily digit $k$.

Let's plot everything with the cluster label $0$. 

In [ ]:
# Get the cluster 0 samples
digits0 = data[kmeans.labels_ == 0]

fig, axes = plt.subplots(12, 20, figsize=(10, 6))
# Turn off everything except the images and border
for ax in axes.flatten():
    ax.tick_params(
        axis='both',
        which='both',
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False
    )
# Plot the images
for i, ax in enumerate(axes.flatten()):
    if i < len(digits0):
        ax.imshow(digits0[i].reshape(8, 8), cmap='gray_r')

plt.show()

We knew how many clusters we wanted because we know there are just 10 digits. 

But needing to declare the number of clusters at the outset makes this a bit awkward.

- For example, the algorithm cannot learn from the data how many clusters there should be.

Well... this isn't exactly true.

There are *method and heuristics* to try to determine the optimal number of clusters.
- What is "optimal"? 🤷

There are some standard notions of optimal, but they are guidelines more than anything.

A metric for this is called *inertia*

In [ ]:
kmeans.inertia_

For each sample
- we compute the squared distance to its centroid
- then we sum up the values. 

One heuristic is to make an "elbow plot" -- at the one hopes it resembles an elbow.

- Pick some candidate $k$ values, maybe `range(1, 26)`, and plot the inertia for each.
- If the plot sharply decreases as $k$ increases and then suddenly plateaus, you are in luck!


This does *not* happen for the digits. 

In [ ]:
inerts = []
ks = range(1,21)
for k in ks:
    kmeans_model = KMeans(n_clusters=k)
    kmeans_model.fit(reduced_data)          # Try X (iris data)
    inerts.append(kmeans_model.inertia_)

plt.figure(figsize=(10,6))
plt.grid()
plt.plot(ks, inerts, 'bx-')
plt.xlabel('$k$')
plt.ylabel('Inertia')
plt.title('The Elbow Method showing the optimal $k$')
plt.xticks(ks)
plt.show()

There are other ways to improve the elbow plot, but it still only sometimes works.

### Upsides and downsides of $k$-means

Let's load a helper function to demonstrate $k$-means.

In [ ]:
# Helper function to plot the Voronoi diagram
def Voronoi(data, kmeans=None):
    x_min = (data.T)[0].min()
    x_max = (data.T)[0].max()
    y_min = (data.T)[1].min()
    y_max = (data.T)[1].max()
    ma = max(x_max, y_max)
    mi = min(x_min, y_min)
    x_min -= (ma - mi)/20
    y_min -= (ma - mi)/20
    x_max += (ma - mi)/20
    y_max += (ma - mi)/20

    plt.figure(1)
    plt.clf()
    if kmeans:
        incr = 0.01
        xx, yy = np.meshgrid(
            np.arange(x_min, x_max, incr), 
            np.arange(y_min, y_max, incr)
        )
        Z = np.c_[xx.ravel(), yy.ravel()]
        kmeans.fit(Z)
        L = kmeans.predict(Z)
        L = L.reshape(xx.shape)
        plt.imshow(
            L,
            interpolation="nearest",
            extent=(xx.min(), xx.max(), yy.min(), yy.max()),
            cmap=plt.cm.Paired,
            aspect="auto",
            origin="lower",
        )
    plt.tick_params(
        left = False, 
        right = False, 
        labelleft = False, 
        labelbottom = False, 
        bottom = False
    )
    plt.scatter(data[:,0], data[:,1], c="black", alpha=0.66, s=25)    
    if kmeans:
        cents = kmeans.cluster_centers_
        plt.scatter(cents[:,0], cents[:,1], c="white", marker="x", linewidths=2, s=100)
    return plt

There are 4 data sets for this demonstration:
- `three_clusters_clear.csv`
- `three_clusters_vague.csv`
- `two_clusters_unbalanced.csv`
- `impossible.csv`

We'll look at the first and third. 

Let's plot the first data set.

In [ ]:
data = pd.read_csv("data/three_clusters_clear.csv").to_numpy()
Voronoi(data).show()

Now let's produce a clustering.

In [ ]:
kmeans = KMeans(n_clusters=3)
Voronoi(data, kmeans).show()

One more data set to consider.

In [ ]:
data = pd.read_csv("data/two_clusters_unbalanced.csv").to_numpy()
Voronoi(data).show()

In [ ]:
kmeans = KMeans(n_clusters=2)
Voronoi(data, kmeans).show()

$k$-means is not the only clustering algorithm out there. 

There are many more, even ones that are fundamentally different:
- heirarchical clustering
- topological clustering

## Recap

There are two types of machine learning:
- supervised
- unsupervised

This corresponds to whether or not the data is labeled.

While one particular tool might not be useful in a specific situation, it is important to be familiar with the whole toolbox.

We saw algorithms and examples of 
- decision trees
- linear regression
- principal component analysis
- $k$-means

## Exercises

1. Run through the Iris example with:
	- no scaler
	- the `MinMaxScaler`

   what differences do you see? How do the correlation matrices differ? You can build a correlation matrix with the `corr` method: e.g. `df.corr()`.

2. Run a $k$-means clustering on `three_clusters_vague.csv` and `impossible.csv`. Feel free to use the helper function `Voronoi(data, kmeans=None)`, where `data` a NumPy array of your data and `kmeans` is the `kmeans` model. If no model is given, a standard plot of the data is returned.